In [2]:
%%javascript
/**********************************************************************************************
Known Mathjax Issue with Chrome - a rounding issue adds a border to the right of mathjax markup
https://github.com/mathjax/MathJax/issues/1300
A quick hack to fix this based on stackoverflow discussions: 
http://stackoverflow.com/questions/34277967/chrome-rendering-mathjax-equations-with-a-trailing-vertical-line
**********************************************************************************************/

$('.math>span').css("border-left-color","transparent")

<IPython.core.display.Javascript object>

In [68]:
%reload_ext autoreload
%autoreload 2

# DATASCI W261 - Machine Learning At Scale
## Assignment - Week 09
---
__Name:__  Carlos Eduardo Rodriguez Castillo   
__Class:__ W261 - Section 2     
__Email:__  cerodriguez@ischool.berkeley.edu     
__Week:__   09 


---
### Instructions

Due by 07/17/2016

[Submission Link - Google Form](https://docs.google.com/forms/d/1ZOr9RnIe_A06AcZDB6K1mJN4vrLeSmS2PD6Xm3eOiis/viewform?usp=send_form) 

### Documents:
* IPython Notebook, published and viewable online.
* PDF export of IPython Notebook.
    
### Useful References

* Data-intensive text processing with MapReduce. San Rafael, CA: Morgan & Claypool Publishers. Chapter 5. 


---

<h2 style="color:red">HW 9 Dataset</h2>

Note that all referenced files life in the enclosing directory. [Checkout the Data subdirectory on Dropbox](https://www.dropbox.com/sh/2c0k5adwz36lkcw/AAAAKsjQfF9uHfv-X9mCqr9wa?dl=0) or the AWS S3 buckets (details contained each question). 

<h2 style="color:red"> HW 9.0: Short answer questions </h2>

__ What is PageRank and what is it used for in the context of web search?__

##### ANSWER:

PageRank is an algorithm for ranking the relevance or popularity of nodes in a graph based on the steady-state probability distribution of the nodes in the graph.

PageRank is the backbone of modern web search. Larry Page and Sergei Brin used PageRank as the core of the Google search engine which critically aided them in making Google the most popular web search solution in the world.

<hr>

__ What modifications have to be made to the webgraph in order to leverage the machinery of Markov Chains to compute the Steady State Distibution? __

In order to leverage the machinery of Markov Chains to compute the steady-state distribution

need to deal with dangling node (no outlinks; needs to be processed for the purposes of not having ) and teleportation

<hr>

__ OPTIONAL: In topic-specific pagerank, how can we ensure that the irreducible property is satifsied? (HINT: see HW9.4) __



<hr>


<h2 style="color:red"> HW 9.1: MRJob implementation of basic PageRank </h2>

Write a basic MRJob implementation of the iterative PageRank algorithm that takes sparse adjacency lists as input (as explored in HW 7).

Make sure that you implementation utilizes teleportation (1-damping/the number of nodes in the network), and further, distributes the mass of dangling nodes with each iteration so that the output of each iteration is correctly normalized (sums to 1).


[NOTE: The PageRank algorithm assumes that a random surfer (walker), starting from a random web page, chooses the next page to which it will move by clicking at random, with probability d,one of the hyperlinks in the current page. This probability is represented by a so-called *damping factor* d, where d ∈ (0, 1). Otherwise, with probability (1 − d), the surfer jumps to any web page in the network. If a page is a dangling end, meaning it has no outgoing hyperlinks, the random surfer selects an arbitrary web page from a uniform distribution and “teleports” to that page]


As you build your code, use the test data:

> s3://ucb-mids-mls-networks/PageRank-test.txt

Or under the Data Subfolder for HW7 on Dropbox with the same file name. 
> Dropbox: https://www.dropbox.com/sh/2c0k5adwz36lkcw/AAAAKsjQfF9uHfv-X9mCqr9wa?dl=0

with teleportation parameter set to 0.15 (1-d, where d, the damping factor is set to 0.85), and crosscheck your work with the true result, displayed in the first image in the [Wikipedia article](https://en.wikipedia.org/wiki/PageRank)
and here for reference are the corresponding PageRank probabilities:
<pre>

A, 0.033
B, 0.384
C, 0.343
D, 0.039
E, 0.081
F, 0.039
G, 0.016
H, 0.016
I, 0.016
J, 0.016
K, 0.016

</pre>

<h2 style="color:darkgreen"> HW 9.1 Implementation </h2>

In [7]:
!wget "https://www.dropbox.com/sh/2c0k5adwz36lkcw/AACf_33TeM7rKfEhjEoNmMCaa/PageRank-test_indexed.txt?dl=0#" \
-O /home/cloudera/w261/HW9/data/toy?example.txt

--2016-07-13 15:29:36--  https://www.dropbox.com/sh/2c0k5adwz36lkcw/AACf_33TeM7rKfEhjEoNmMCaa/PageRank-test_indexed.txt?dl=0
Resolving www.dropbox.com... 162.125.4.1
Connecting to www.dropbox.com|162.125.4.1|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://dl.dropboxusercontent.com/content_link/dEcSLmCwpZqwbtawvqUGR1Wc2pfe4C9n5O5zKQqFGi7U2MlOxGfUnW8iMgvJiHh7/file [following]
--2016-07-13 15:29:36--  https://dl.dropboxusercontent.com/content_link/dEcSLmCwpZqwbtawvqUGR1Wc2pfe4C9n5O5zKQqFGi7U2MlOxGfUnW8iMgvJiHh7/file
Resolving dl.dropboxusercontent.com... 108.160.173.5
Connecting to dl.dropboxusercontent.com|108.160.173.5|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 168 [text/plain]
Saving to: “/home/cloudera/w261/HW9/data/toy?example.txt”

100%[======================================>] 168         --.-K/s   in 0s      

2016-07-13 15:29:38 (12.8 MB/s) - “/home/cloudera/w261/HW9/data/toy?example.txt” saved [168/168]



In [27]:
!cat /home/cloudera/w261/HW9/data/toy/example.txt

2	{'3': 1}
3	{'2': 1}
4	{'1': 1, '2': 1}
5	{'4': 1, '2': 1, '6': 1}
6	{'2': 1, '5': 1}
7	{'2': 1, '5': 1}
8	{'2': 1, '5': 1}
9	{'2': 1, '5': 1}
10	{'5': 1}
11	{'5': 1}


In [102]:
%%writefile PageRank_graph_prep.py
#!/usr/bin/env python

from mrjob.job import MRJob
from mrjob.step import MRStep
import sys
import os
import ast
from mrjob.protocol import RawValueProtocol
from mrjob.protocol import RawProtocol

class MRpageRankGraphPrep(MRJob):
    

    def steps(self):
        JOBCONF2 = {
              'mapred.output.key.comparator.class':'org.apache.hadoop.mapred.lib.KeyFieldBasedComparator',
              'mapred.text.key.comparator.options': '-k1n -k2nr',
              'mapred.reduce.tasks': '1'
        }
        return [MRStep(jobconf=JOBCONF2,
                    mapper=self.mapper, 
                    reducer=self.reducer)
                ]
    
    def mapper(self, _, line):
        line = line.strip()
        key, value = line.split("\t")
        neighbors = ast.literal_eval(value)
        neighbors_string = ""
        for n in neighbors.keys():
            if neighbors_string == "":
                neighbors_string = n
            else:
                neighbors_string = neighbors_string+","+n
        yield int(key), neighbors_string+","+"1"
        
    def reducer(self, key, values):
        for v in values:
            yield key, v
    
if __name__ == "__main__":
    MRpageRankGraphPrep.run()

Overwriting PageRank_graph_prep.py


In [103]:
!chmod a+x PageRank_graph_prep.py

In [104]:
%reload_ext autoreload
%autoreload 2

!./PageRank_graph_prep.py -r hadoop /home/cloudera/w261/HW9/data/toy/example_1.txt

Using configs in /home/cloudera/.mrjob.conf
Creating temp directory /tmp/PageRank_graph_prep.cloudera.20160715.012934.416849
Looking for hadoop binary in $PATH...
Found hadoop binary: /usr/bin/hadoop
Using Hadoop version 2.6.0
Copying local files to hdfs:///user/cloudera/tmp/mrjob/PageRank_graph_prep.cloudera.20160715.012934.416849/files/...
Looking for Hadoop streaming jar in /home/hadoop/contrib...
Looking for Hadoop streaming jar in /usr/lib/hadoop-mapreduce...
Found Hadoop streaming jar: /usr/lib/hadoop-mapreduce/hadoop-streaming.jar
Detected hadoop configuration property names that do not match hadoop version 2.6.0:
The have been translated as follows
 mapred.output.key.comparator.class: mapreduce.job.output.key.comparator.class
mapred.reduce.tasks: mapreduce.job.reduces
mapred.text.key.comparator.options: mapreduce.partition.keycomparator.options
Running step 1 of 1...
  mapred.output.key.comparator.class is deprecated. Instead, use mapreduce.job.output.key.comparator.class
  map

In [306]:
%%writefile PageRank_graph_prep_1.py
#!/usr/bin/env python

from mrjob.job import MRJob
from mrjob.step import MRStep
import sys
import os
import ast
from mrjob.protocol import RawValueProtocol
from mrjob.protocol import RawProtocol

class MRpageRankGraphPrep(MRJob):
    
    num_nodes = 0
    
    def step(self):
        
        JOB_CONF_STEP = {
            'mapred.output.key.comparator.class':'org.apache.hadoop.mapred.lib.KeyFieldBasedComparator',
            'mapred.text.key.comparator.options': '-k1n -k2nr',
            'mapred.reduce.tasks': '1'
        }
        
        return [
            MRStep(jobconf=JOB_CONF_STEP,
                   mapper=self.mapper,
                   reducer=self.reducer)
        ]
    
    def mapper(self, _, line):
        line = line.strip()
        key, value = line.split("\t")
        neighbors = ast.literal_eval(value)
        neighbors_array = []
        for n in neighbors.keys():
#             if neighbors_string == "":
            neighbors_array.append(n)
#             else:
#                 neighbors_string = neighbors_string+","+n
            yield "*"+n, "*"
            yield n, "-"
        yield "*"+key, "*"
        yield key, neighbors_array
        
    def reducer(self, key, values):
        neighbors = []
        if key[0] == "*":
#             self.increment_counter("Nodes","Num_nodes",1)
            self.num_nodes += 1
            sys.stderr.write("The number of nodes so far is %d\n"%self.num_nodes)
        else:
            for v in values:
                if neighbors == "":
                    if v != "-":
                        neighbors = v
                else:
                    if v != "-":
                        neighbors.extend(v)
            yield key, (neighbors, 1.0/float(self.num_nodes))
#         for v in values:
#             yield key, v[0]
        
if __name__ == "__main__":
    MRpageRankGraphPrep.run()

Overwriting PageRank_graph_prep_1.py


In [207]:
!chmod a+x PageRank_graph_prep_1.py

In [119]:
!./PageRank_graph_prep_1.py /home/cloudera/w261/HW9/data/toy/example_1.txt

Using configs in /home/cloudera/.mrjob.conf
Creating temp directory /tmp/PageRank_graph_prep_1.cloudera.20160715.175947.910971
Running step 1 of 1...
Traceback (most recent call last):
  File "./PageRank_graph_prep_1.py", line 60, in <module>
    MRpageRankGraphPrep.run()
  File "/home/cloudera/anaconda2/lib/python2.7/site-packages/mrjob/job.py", line 430, in run
    mr_job.execute()
  File "/home/cloudera/anaconda2/lib/python2.7/site-packages/mrjob/job.py", line 448, in execute
    super(MRJob, self).execute()
  File "/home/cloudera/anaconda2/lib/python2.7/site-packages/mrjob/launch.py", line 160, in execute
    self.run_job()
  File "/home/cloudera/anaconda2/lib/python2.7/site-packages/mrjob/launch.py", line 230, in run_job
    runner.run()
  File "/home/cloudera/anaconda2/lib/python2.7/site-packages/mrjob/runner.py", line 473, in run
    self._run()
  File "/home/cloudera/anaconda2/lib/python2.7/site-packages/mrjob/sim.py", line 185, in _run
    self._invoke_step(step_num, 'reducer'

In [235]:
!mkdir -p /home/cloudera/w261/HW9/data/toy_prepared
!./PageRank_graph_prep_1.py -r hadoop /home/cloudera/w261/HW9/data/toy/example_1.txt \
> /home/cloudera/w261/HW9/data/toy_prepared/input_1.txt

Using configs in /home/cloudera/.mrjob.conf
Creating temp directory /tmp/PageRank_graph_prep_1.cloudera.20160716.044352.810563
Looking for hadoop binary in $PATH...
Found hadoop binary: /usr/bin/hadoop
Using Hadoop version 2.6.0
Copying local files to hdfs:///user/cloudera/tmp/mrjob/PageRank_graph_prep_1.cloudera.20160716.044352.810563/files/...
Looking for Hadoop streaming jar in /home/hadoop/contrib...
Looking for Hadoop streaming jar in /usr/lib/hadoop-mapreduce...
Found Hadoop streaming jar: /usr/lib/hadoop-mapreduce/hadoop-streaming.jar
Running step 1 of 1...
  packageJobJar: [] [/usr/jars/hadoop-streaming-2.6.0-cdh5.7.0.jar] /tmp/streamjob4556544410363699565.jar tmpDir=null
  Connecting to ResourceManager at quickstart.cloudera/10.0.2.15:8032
  Connecting to ResourceManager at quickstart.cloudera/10.0.2.15:8032
  Total input paths to process : 1
  number of splits:2
  Submitting tokens for job: job_1468516795796_0057
  Submitted application application_1468516795796_0057
  The ur

In [308]:
%%writefile PageRank_graph_prep_2.py
#!/usr/bin/env python

from mrjob.job import MRJob
from mrjob.step import MRStep
import sys
import os
import ast
from mrjob.protocol import RawValueProtocol
from mrjob.protocol import RawProtocol

class MRpageRankGraphPrep(MRJob):
    
    num_nodes = 0
    
    def step(self):
        
        JOB_CONF_STEP = {
            'mapred.output.key.comparator.class':'org.apache.hadoop.mapred.lib.KeyFieldBasedComparator',
            'mapred.text.key.comparator.options': '-k1n -k2nr',
            'mapred.reduce.tasks': '1'
        }
        
        return [
            MRStep(#jobconf=JOB_CONF_STEP,
                   mapper=self.mapper,
                   reducer=self.reducer)
        ]
    
    def mapper(self, _, line):
        line = line.strip()
        key, value = line.split("\t")
        neighbors = ast.literal_eval(value)
        neighbors_array = []
        for n in neighbors.keys():
            neighbors_array.append(n)
            yield n, "-"
        yield key, neighbors_array
        
    def reducer(self, key, values):
        neighbors = []
        self.increment_counter("Nodes","Num_nodes",1)
        for v in values:
            if neighbors == "":
                if v != "-":
                    neighbors = v
            else:
                if v != "-":
                    neighbors.extend(v)
        yield key, (neighbors, 1.0)
        
if __name__ == "__main__":
    MRpageRankGraphPrep.run()

Overwriting PageRank_graph_prep_2.py


In [309]:
!chmod a+x PageRank_graph_prep_2.py

In [310]:
!./PageRank_graph_prep_2.py -r hadoop /home/cloudera/w261/HW9/data/toy/example_1.txt \
> /home/cloudera/w261/HW9/data/toy_prepared_graph_input/input.txt

Using configs in /home/cloudera/.mrjob.conf
Creating temp directory /tmp/PageRank_graph_prep_2.cloudera.20160716.221234.792099
Looking for hadoop binary in $PATH...
Found hadoop binary: /usr/bin/hadoop
Using Hadoop version 2.6.0
Copying local files to hdfs:///user/cloudera/tmp/mrjob/PageRank_graph_prep_2.cloudera.20160716.221234.792099/files/...
Looking for Hadoop streaming jar in /home/hadoop/contrib...
Looking for Hadoop streaming jar in /usr/lib/hadoop-mapreduce...
Found Hadoop streaming jar: /usr/lib/hadoop-mapreduce/hadoop-streaming.jar
Running step 1 of 1...
  packageJobJar: [] [/usr/jars/hadoop-streaming-2.6.0-cdh5.7.0.jar] /tmp/streamjob3587476104058141132.jar tmpDir=null
  Connecting to ResourceManager at quickstart.cloudera/10.0.2.15:8032
  Connecting to ResourceManager at quickstart.cloudera/10.0.2.15:8032
  Total input paths to process : 1
  number of splits:2
  Submitting tokens for job: job_1468516795796_0062
  Submitted application application_1468516795796_0062
  The ur

In [314]:
!cat /home/cloudera/w261/HW9/data/toy/example_1.txt

2	{'3': 1}
3	{'2': 1}
4	{'1': 1, '2': 1}
5	{'4': 1, '2': 1, '6': 1}
6	{'2': 1, '5': 1}
7	{'2': 1, '5': 1}
8	{'2': 1, '5': 1}
9	{'2': 1, '5': 1}
10	{'5': 1}
11	{'5': 1}


In [318]:
## first we spool up a cluster
!mrjob create-cluster \
--max-hours-idle 1 \
--aws-region=us-west-2 -c ~/.mrjob.conf

Using s3://mrjob-4b78edd1b5baad75/tmp/ as our temp dir on S3
Creating persistent cluster to run several jobs in...
Creating temp directory /tmp/no_script.cloudera.20160716.223709.782630
Copying local files to s3://mrjob-4b78edd1b5baad75/tmp/no_script.cloudera.20160716.223709.782630/files/...
j-1I28YD5XHICD3


In [353]:
%reload_ext autoreload
%autoreload 2

from numpy import random,array
# from PageRank_step1 import MRpageRank1
from PageRank_graph_prep_2 import MRpageRankGraphPrep
from PageRank_step2 import MRpageRank2
import ast
import os

!rm -r /home/cloudera/w261/HW9/data/toy_prepared_graph_input
!rm -r /home/cloudera/w261/HW9/data/toy_prepared_graph_output
!mkdir /home/cloudera/w261/HW9/data/toy_prepared_graph_input
!rm -r /home/cloudera/w261/HW9/data/toy_directed_graph_input

mr_job = MRpageRankGraphPrep(args=["/home/cloudera/w261/HW9/data/toy/example_1.txt",
                                    "--output-dir",
                                    "/home/cloudera/w261/HW9/data/toy_prepared_graph_input"])

with mr_job.make_runner() as runner: 
        runner.run()
        counters = runner.counters()[0]
        num_nodes = counters['Nodes']['Num_nodes']

print "NUM NODES IS %d"%num_nodes

i = 0
while(1):

    if i == 0:
        first_run = 1
    else:
        first_run = 0
    mr_job = MRpageRank2(args=["/home/cloudera/w261/HW9/data/toy_prepared_graph_input",
                               "--alpha",0.15,
                               "--num-nodes", str(num_nodes),
                               "--first-run", str(first_run),
                                         "--output-dir",
                               "/home/cloudera/w261/HW9/data/toy_prepared_graph_output"])
    
    print "iteration "+str(i)+" of Pagerank algorithm"

    with mr_job.make_runner() as runner: 
        runner.run()
    i = i + 1

    !rm -r /home/cloudera/w261/HW9/data/toy_prepared_graph_input
    !cp -rT /home/cloudera/w261/HW9/data/toy_prepared_graph_output /home/cloudera/w261/HW9/data/toy_prepared_graph_input

    if i >=40:
        break
!cat /home/cloudera/w261/HW9/data/toy_prepared_graph_output/*

rm: cannot remove `/home/cloudera/w261/HW9/data/toy_directed_graph_input': No such file or directory
NUM NODES IS 11
iteration 0 of Pagerank algorithm
iteration 1 of Pagerank algorithm
iteration 2 of Pagerank algorithm
iteration 3 of Pagerank algorithm
iteration 4 of Pagerank algorithm
iteration 5 of Pagerank algorithm
iteration 6 of Pagerank algorithm
iteration 7 of Pagerank algorithm
iteration 8 of Pagerank algorithm
iteration 9 of Pagerank algorithm
iteration 10 of Pagerank algorithm
iteration 11 of Pagerank algorithm
iteration 12 of Pagerank algorithm
iteration 13 of Pagerank algorithm
iteration 14 of Pagerank algorithm
iteration 15 of Pagerank algorithm
iteration 16 of Pagerank algorithm
iteration 17 of Pagerank algorithm
iteration 18 of Pagerank algorithm
iteration 19 of Pagerank algorithm
iteration 20 of Pagerank algorithm
iteration 21 of Pagerank algorithm
iteration 22 of Pagerank algorithm
iteration 23 of Pagerank algorithm
iteration 24 of Pagerank algorithm
iteration 25 of Pa

In [350]:
import ast

output_dir = "/home/cloudera/w261/HW9/data/toy_prepared_graph_output/"

for name in os.listdir(input_dir):
    with open(input_dir+name) as f:
        for line in f:
            key, value = line.split("\t")
            value = ast.literal_eval(value)
            PR = value[1]
            print "node: %s\tpageRank: %.3f"%(key,PR)

node: "5"	pageRank: 0.081
node: "6"	pageRank: 0.039
node: "7"	pageRank: 0.016
node: "8"	pageRank: 0.016
node: "9"	pageRank: 0.016
node: "1"	pageRank: 0.033
node: "10"	pageRank: 0.016
node: "11"	pageRank: 0.016
node: "2"	pageRank: 0.384
node: "3"	pageRank: 0.343
node: "4"	pageRank: 0.039


In [355]:
%reload_ext autoreload
%autoreload 2

from numpy import random,array
from PageRank_step2 import MRpageRank2
from PageRank_graph_prep_2 import MRpageRankGraphPrep
import ast
import os

!aws s3 rm s3://cerc-w261/HW9/toy/prepared_graph_input --recursive --quiet
!aws s3 rm s3://cerc-w261/HW9/toy/prepared_graph_output --recursive --quiet

mr_job = MRpageRankGraphPrep(args=["-r","emr",
                                    "s3://cerc-w261/HW9/toy/raw_input",
                                   "--cluster-id=j-1I28YD5XHICD3",
                                   "--aws-region=us-west-2",
                                    "--output-dir",
                                    "s3://cerc-w261/HW9/toy/prepared_graph_input"])

with mr_job.make_runner() as runner: 
        runner.run()
        counters = runner.counters()[0]
        num_nodes = counters['Nodes']['Num_nodes']
print "NUM NODES IS"
print num_nodes
i = 0
while(1):

    if i == 0:
        first_run = 1
    else:
        first_run = 0
    mr_job = MRpageRank2(args=["-r","emr",
                               "s3://cerc-w261/HW9/toy/prepared_graph_input",
                               "--alpha", str(0.15),
                               "--num-nodes", str(num_nodes),
                               "--first-run", str(first_run),
                                         "--output-dir",
                               "s3://cerc-w261/HW9/toy/prepared_graph_output",
                              "--cluster-id=j-1I28YD5XHICD3",
                              "--aws-region=us-west-2"])
    
    print "iteration "+str(i)+" of Pagerank algorithm"

    with mr_job.make_runner() as runner: 
        runner.run()
    i = i + 1

    !aws s3 rm s3://cerc-w261/HW9/toy/prepared_graph_input/ --recursive --quiet
    !aws s3 cp s3://cerc-w261/HW9/toy/prepared_graph_output s3://cerc-w261/HW9/toy/prepared_graph_input --recursive --quiet
    !aws s3 rm s3://cerc-w261/HW9/toy/prepared_graph_output --recursive --quiet

    if i >=40:
        break

NUM NODES IS
11
iteration 0 of Pagerank algorithm
iteration 1 of Pagerank algorithm
iteration 2 of Pagerank algorithm
iteration 3 of Pagerank algorithm
iteration 4 of Pagerank algorithm
iteration 5 of Pagerank algorithm
iteration 6 of Pagerank algorithm
iteration 7 of Pagerank algorithm
iteration 8 of Pagerank algorithm
iteration 9 of Pagerank algorithm
iteration 10 of Pagerank algorithm
iteration 11 of Pagerank algorithm
iteration 12 of Pagerank algorithm
iteration 13 of Pagerank algorithm
iteration 14 of Pagerank algorithm
iteration 15 of Pagerank algorithm
iteration 16 of Pagerank algorithm
iteration 17 of Pagerank algorithm
iteration 18 of Pagerank algorithm
iteration 19 of Pagerank algorithm
iteration 20 of Pagerank algorithm
iteration 21 of Pagerank algorithm
iteration 22 of Pagerank algorithm
iteration 23 of Pagerank algorithm
iteration 24 of Pagerank algorithm
iteration 25 of Pagerank algorithm
iteration 26 of Pagerank algorithm
iteration 27 of Pagerank algorithm
iteration 28 o

In [236]:
!cat  /home/cloudera/w261/HW9/data/toy_prepared/input_1.txt

"1"	[[], 0.090909090909090912]
"10"	[["5"], 0.090909090909090912]
"11"	[["5"], 0.090909090909090912]
"2"	[["3"], 0.090909090909090912]
"3"	[["2"], 0.090909090909090912]
"4"	[["1", "2"], 0.090909090909090912]
"5"	[["2", "4", "6"], 0.090909090909090912]
"6"	[["2", "5"], 0.090909090909090912]
"7"	[["2", "5"], 0.090909090909090912]
"8"	[["2", "5"], 0.090909090909090912]
"9"	[["2", "5"], 0.090909090909090912]


In [343]:
%%writefile PageRank_step2.py
#!/usr/bin/env python

from mrjob.job import MRJob
from mrjob.step import MRStep
import sys
import os
import ast
from mrjob.protocol import RawValueProtocol
from mrjob.protocol import RawProtocol

class MRpageRank2(MRJob):
    
    ## Hardcoding the number of nodes in the graph
    ## these are substituted with runtime parameters
    N = 1
    alpha = 0.0
    first_run = 0
    
    ## function permits us to pass an arbitrary alpha
    ## to our mrjob
    def configure_options(self):
        super(MRpageRank2, self).configure_options()
        self.add_passthrough_option('--alpha', type='float', default=0.1, help='The damping factor; must be a float!')
        self.add_passthrough_option('--first-run', type='int', default=0, help='Indicates whether we are dealing with the first run of algo')
        self.add_passthrough_option('--num-nodes', type='int', default=1, help='The number of nodes in the graph; must be an int!')
    
    def steps(self):
        return [
            MRStep(
            mapper_init=self.mapper_init,
            mapper=self.mapper,
            reducer_init=self.reducer_init,
            reducer=self.reducer),
            MRStep(
                reducer_init=self.reducer2_init,
            reducer=self.reducer2)
        ]
    
    def mapper_init(self):
        self.N = self.options.num_nodes
        self.first_run = self.options.first_run
    
    def mapper(self, _, line):
        line = line.strip()
        key, value = line.split("\t")
        key = key.strip('"')
        value = ast.literal_eval(value)
        neighbors = value[0]
#         print "THESE ARE THE NEIGHBORS"
#         print neighbors
        if self.first_run == 1:
            PR = float(value[1]) / float(self.N)
        else:
            PR = float(value[1])
        if neighbors == []:
            yield "*dangling", ("-", float(PR))
            yield key, (neighbors, 0.0)
        else:
#             Before, neighbors where a comma-separated string
#             neighbors = neighbors.split(",")
#             Given the literal_eval above, this one may not be necessary under the new world
#             neighbors = ast.literal_eval(neighbors)
            num_nieghbors = len(neighbors)
            for n in neighbors:
                yield n, ("-",float(PR)/float(num_nieghbors))
            yield key, (neighbors, 0.0)
    
    def reducer_init(self):
        self.N = self.options.num_nodes
    
    def reducer(self, key, values):
        dangling_mass = 0.0
        if key == "*dangling":
            for v in values:
                dangling_mass += v[1]
            for i in range(1,self.N + 1):
                yield str(i), ("-",float(dangling_mass)/float(self.N))
        else:
            new_PR = 0.0
            neighbors = ""
            for v in values:
                new_PR += v[1]
                if v[0] != "-":
                    neighbors = v[0]
            yield key, (neighbors, new_PR)
    
    def reducer2_init(self):
        self.alpha = self.options.alpha
        self.N = self.options.num_nodes
        
    def reducer2(self, key, values):
        new_PR = 0.0
        neighbors = ""
#         values = ast.literal_eval(values)
        for v in values:
            new_PR += float(v[1])
            if v[0] != "-":
                neighbors = v[0]
        new_PR = self.alpha * 1.00/float(self.N) + (1 - self.alpha) * new_PR
        yield key, (neighbors, new_PR)
#         for v in values:
#             yield v, None
        
if __name__ == "__main__":
    MRpageRank2.run()

Overwriting PageRank_step2.py


In [344]:
!chmod a+x PageRank_step2.py

In [ ]:
%%writefile PageRank_step3.py
#!/usr/bin/env python

from mrjob.job import MRJob
from mrjob.step import MRStep
import sys
import os
import ast
from mrjob.protocol import RawValueProtocol
from mrjob.protocol import RawProtocol

class MRpageRank3(MRJob):
    
    ## Hardcoding the number of nodes in the graph
    ## these are substituted with runtime parameters
    N = 1
    alpha = 0.0
    first_run = 0
    dangling_mass = 0.0
    
    ## function permits us to pass an arbitrary alpha
    ## to our mrjob
    def configure_options(self):
        super(MRpageRank3, self).configure_options()
        self.add_passthrough_option('--alpha', type='float', default=0.1, help='The damping factor; must be a float!')
        self.add_passthrough_option('--first-run', type='int', default=0, help='Indicates whether we are dealing with the first run of algo')
        self.add_passthrough_option('--num-nodes', type='int', default=1, help='The number of nodes in the graph; must be an int!')
    
    def steps(self):
        return [
            MRStep(
            mapper_init=self.mapper_init,
            mapper=self.mapper,
            reducer_init=self.reducer_init,
            reducer=self.reducer),
            MRStep(
                reducer_init=self.reducer2_init,
            reducer=self.reducer2)
        ]
    
    def mapper_init(self):
        self.N = self.options.num_nodes
        self.first_run = self.options.first_run
    
    def mapper(self, _, line):
        line = line.strip()
        key, value = line.split("\t")
        key = key.strip('"')
        value = ast.literal_eval(value)
        neighbors = value[0]
#         print "THESE ARE THE NEIGHBORS"
#         print neighbors
        if self.first_run == 1:
            PR = float(value[1]) / float(self.N)
        else:
            PR = float(value[1])
        if neighbors == []:
            yield "*dangling", ("-", float(PR))
            yield key, (neighbors, 0.0)
        else:
#             Before, neighbors where a comma-separated string
#             neighbors = neighbors.split(",")
#             Given the literal_eval above, this one may not be necessary under the new world
#             neighbors = ast.literal_eval(neighbors)
            num_nieghbors = len(neighbors)
            for n in neighbors:
                yield n, ("-",float(PR)/float(num_nieghbors))
            yield key, (neighbors, 0.0)
    
    def reducer_init(self):
        self.N = self.options.num_nodes
    
    def reducer(self, key, values):
        dangling_mass = 0.0
        if key == "*dangling":
            for v in values:
                dangling_mass += v[1]
            self.increment_counter("dangling_nodes","mass", dangling_mass)
            # Commenting out the below as I am attempting to pass
            # the dangling mass through a counter
#             for i in range(1,self.N + 1):
#                 yield str(i), ("-",float(dangling_mass)/float(self.N))
        else:
            new_PR = 0.0
            neighbors = ""
            for v in values:
                new_PR += v[1]
                if v[0] != "-":
                    neighbors = v[0]
            yield key, (neighbors, new_PR)
    
    def reducer2_init(self):
        self.alpha = self.options.alpha
        self.N = self.options.num_nodes
        self.dangling_mass = self.counters()[0]['dangling_nodes']['mass']
        
    def reducer2(self, key, values):
        new_PR = 0.0
        neighbors = ""
        for v in values:
            new_PR += float(v[1])
            if v[0] != "-":
                neighbors = v[0]
        # Here we distribute the dangling mass that was captured through the counter
        new_PR = self.alpha * 1.00/float(self.N) + (1 - self.alpha) * (new_PR  + (self.dangling_mass / float(self.N)))
        yield key, (neighbors, new_PR)
        
if __name__ == "__main__":
    MRpageRank3.run()

In [ ]:
self.increment_counter("Nodes","Num_nodes",1)

In [295]:
%%writefile PageRank_step1.py
#!/usr/bin/env python

from mrjob.job import MRJob
from mrjob.step import MRStep
import sys
import os
import ast
from mrjob.protocol import RawValueProtocol
from mrjob.protocol import RawProtocol

## Check k-means convergence
def stop_criterion(nodes_old, nodes_new,T):
    
    ## produce array of coordinates for old and new centroids
    oldvalue = list(chain(*centroid_points_old))
    newvalue = list(chain(*centroid_points_new))
    
    ## compute the absolute difference between the new
    ## and old values for each coordinate point
    Diff = [abs(a-b) for a, b in zip(oldvalue, newvalue)]
    
    Flag = True
    
    ## Iterate through all parameters' absolute differences
    for i in Diff:
        
        ## if any of the dimensions' differences is above the convergence
        ## threshold, then fail the convergence check
        if(i>T):
            ## False flag entails failure to converge
            Flag = False
            break
    return Flag

class MRpageRank1(MRJob):
    
    ## Hardcoding the number of nodes in the graph
    ## these are substituted with runtime parameters
    N = 1
    alpha = 0.0
    
    ## function permits us to pass an arbitrary alpha
    ## to our mrjob
    def configure_options(self):
        super(MRpageRank1, self).configure_options()
        self.add_passthrough_option('--alpha', type='float', default=0.1, help='The damping factor; must be a float!')
        self.add_passthrough_option('--num-nodes', type='int', default=1, help='The number of nodes in the graph; must be an int!')
    
    def steps(self):
        return [
            MRStep(
            mapper=self.mapper,
            reducer_init=self.reducer_init,
            reducer=self.reducer),
            MRStep(
                reducer_init=self.reducer2_init,
            reducer=self.reducer2)
        ]
    def mapper(self, _, line):
        line = line.strip()
        key, value = line.split("\t")
        key = key.strip('"')
        value = ast.literal_eval(value)
        neighbors = value[0]
#         print "THESE ARE THE NEIGHBORS"
#         print neighbors
        PR = value[1]
        if neighbors == []:
            yield "*dangling", ("-", float(PR))
            yield key, (neighbors, 0.0)
        else:
#             Before, neighbors where a comma-separated string
#             neighbors = neighbors.split(",")
#             Given the literal_eval above, this one may not be necessary under the new world
#             neighbors = ast.literal_eval(neighbors)
            num_nieghbors = len(neighbors)
            for n in neighbors:
                yield n, ("-",float(PR)/float(num_nieghbors))
            yield key, (neighbors, 0.0)
    
    def reducer_init(self):
        self.N = self.options.num_nodes
    
    def reducer(self, key, values):
        dangling_mass = 0.0
        if key == "*dangling":
            for v in values:
                dangling_mass += v[1]
            for i in range(1,self.N + 1):
                yield str(i), ("-",float(dangling_mass)/float(self.N))
        else:
            new_PR = 0.0
            neighbors = ""
            for v in values:
                new_PR += v[1]
                if v[0] != "-":
                    neighbors = v[0]
            yield key, (neighbors, new_PR)
    
    def reducer2_init(self):
        self.alpha = self.options.alpha
        self.N = self.options.num_nodes
        
    def reducer2(self, key, values):
        new_PR = 0.0
        neighbors = ""
#         values = ast.literal_eval(values)
        for v in values:
            new_PR += float(v[1])
            if v[0] != "-":
                neighbors = v[0]
        new_PR = self.alpha * 1.00/float(self.N) + (1 - self.alpha) * new_PR
        yield key, (neighbors, new_PR)
#         for v in values:
#             yield v, None
        
if __name__ == "__main__":
    MRpageRank1.run()

Overwriting PageRank_step1.py


In [296]:
!chmod a+x PageRank_step1.py

In [297]:
# !cat /home/cloudera/w261/HW9/data/toy_prepared/input_1.txt
num_nodes = 11
!./PageRank_step1.py --alpha=0.15 --num-nodes=$num_nodes \
/home/cloudera/w261/HW9/data/toy_prepared/input_1.txt

Using configs in /home/cloudera/.mrjob.conf
Creating temp directory /tmp/PageRank_step1.cloudera.20160716.162802.547998
Running step 1 of 2...
Running step 2 of 2...
Streaming final output from /tmp/PageRank_step1.cloudera.20160716.162802.547998/output...
"5"	[["2", "4", "6"], 0.32975206611570246]
"6"	[["2", "5"], 0.046418732782369146]
"7"	[["2", "5"], 0.02066115702479339]
"8"	[["2", "5"], 0.02066115702479339]
"9"	[["2", "5"], 0.02066115702479339]
"1"	[[], 0.059297520661157024]
"10"	[["5"], 0.02066115702479339]
"11"	[["5"], 0.02066115702479339]
"2"	[["3"], 0.3168732782369146]
"3"	[["2"], 0.09793388429752066]
"4"	[["1", "2"], 0.046418732782369146]
Removing temp directory /tmp/PageRank_step1.cloudera.20160716.162802.547998...


#### Local PageRank Iterative algorithm - Toy graph

In [298]:
%reload_ext autoreload
%autoreload 2

from numpy import random,array
from PageRank_step1 import MRpageRank1
import ast
import os

## Setting up input and output folders for 
## the iterative algorithm
!rm -r /home/cloudera/w261/HW9/data/toy_prepared_graph_input
!rm -r /home/cloudera/w261/HW9/data/toy_prepared_graph_output
!mkdir /home/cloudera/w261/HW9/data/toy_prepared_graph_input
!rm -r /home/cloudera/w261/HW9/data/toy_directed_graph_input

!./PageRank_graph_prep_1.py -r hadoop /home/cloudera/w261/HW9/data/toy/example_1.txt \
> /home/cloudera/w261/HW9/data/toy_prepared_graph_input/input.txt

# !./PageRank_step1.py \
# /home/cloudera/w261/HW9/data/toy_prepared/input_1.txt \
# > /home/cloudera/w261/HW9/data/toy_prepared_graph_input/input.txt

input_dir = "/home/cloudera/w261/HW9/data/toy_prepared_graph_input/"

for name in os.listdir(input_dir):
    with open(input_dir+name) as f:
        num_nodes = 0
        for line in f:
            num_nodes += 1

i = 0
while(1):
#     frontier_exists = False
#     j = i + 1
    mr_job = MRpageRank1(args=["/home/cloudera/w261/HW9/data/toy_prepared_graph_input",
                               "--alpha",0.15,
                               "--num-nodes", str(num_nodes),
                                         "--output-dir", "/home/cloudera/w261/HW9/data/toy_prepared_graph_output"])
    print "iteration "+str(i)+" of Pagerank algorithm"
    graph = []
    with mr_job.make_runner() as runner: 
        runner.run()
#         for line in runner.stream_output():
#             node, payload = line.split("\t")
#             node = node.strip('"')
#             payload_raw = payload
#             payload = ast.literal_eval(payload)
#             state = payload[2]
#             if state == "Q":
#                 frontier_exists = True
    i = i + 1

    !rm -r /home/cloudera/w261/HW9/data/toy_prepared_graph_input
    !cp -rT /home/cloudera/w261/HW9/data/toy_prepared_graph_output /home/cloudera/w261/HW9/data/toy_prepared_graph_input
#     if not frontier_exists:
#         break
    if i >=50:
        break
!cat /home/cloudera/w261/HW9/data/toy_prepared_graph_output/*

rm: cannot remove `/home/cloudera/w261/HW9/data/toy_directed_graph_input': No such file or directory
Using configs in /home/cloudera/.mrjob.conf
Creating temp directory /tmp/PageRank_graph_prep_1.cloudera.20160716.162826.693499
Looking for hadoop binary in $PATH...
Found hadoop binary: /usr/bin/hadoop
Using Hadoop version 2.6.0
Copying local files to hdfs:///user/cloudera/tmp/mrjob/PageRank_graph_prep_1.cloudera.20160716.162826.693499/files/...
Looking for Hadoop streaming jar in /home/hadoop/contrib...
Looking for Hadoop streaming jar in /usr/lib/hadoop-mapreduce...
Found Hadoop streaming jar: /usr/lib/hadoop-mapreduce/hadoop-streaming.jar
Running step 1 of 1...
  packageJobJar: [] [/usr/jars/hadoop-streaming-2.6.0-cdh5.7.0.jar] /tmp/streamjob7007675728165044674.jar tmpDir=null
  Connecting to ResourceManager at quickstart.cloudera/10.0.2.15:8032
  Connecting to ResourceManager at quickstart.cloudera/10.0.2.15:8032
  Total input paths to process : 1
  number of splits:2
  Submitting t

#### EMR PageRank Iterative algorithm - Toy graph

In [301]:
## first we spool up a cluster
!mrjob create-cluster \
--max-hours-idle 1 \
--aws-region=us-west-2 -c ~/.mrjob.conf

Using s3://mrjob-4b78edd1b5baad75/tmp/ as our temp dir on S3
Creating persistent cluster to run several jobs in...
Creating temp directory /tmp/no_script.cloudera.20160716.165027.210554
Copying local files to s3://mrjob-4b78edd1b5baad75/tmp/no_script.cloudera.20160716.165027.210554/files/...
j-2TFUMR7W2PF1V


In [300]:
!aws s3 cp /home/cloudera/w261/HW9/data/toy/example_1.txt s3://cerc-w261/HW9/toy/raw_input/

upload: ../data/toy/example_1.txt to s3://cerc-w261/HW9/toy/raw_input/example_1.txt


In [302]:
!ls /home/cloudera/w261/HW9/data/toy_prepared_graph_input/

part-00000  part-00001


In [307]:
%reload_ext autoreload
%autoreload 2

!aws s3 rm s3://cerc-w261/HW9/toy/prepared_graph_input --recursive --quiet
!aws s3 rm s3://cerc-w261/HW9/toy/prepared_graph_output --recursive --quiet

# !./PageRank_graph_prep_1.py -r hadoop /home/cloudera/w261/HW9/data/toy/example_1.txt \
# > /home/cloudera/w261/HW9/data/toy_prepared_graph_input/input.txt

!python PageRank_graph_prep_1.py \
-r emr s3://cerc-w261/HW9/toy/raw_input \
    --cluster-id=j-2TFUMR7W2PF1V \
    --aws-region=us-west-2 \
    --output-dir=s3://cerc-w261/HW9/toy/prepared_graph_input \
    --no-output

Using configs in /home/cloudera/.mrjob.conf
Using s3://mrjob-4b78edd1b5baad75/tmp/ as our temp dir on S3
Creating temp directory /tmp/PageRank_graph_prep_1.cloudera.20160716.171336.184035
Copying local files to s3://mrjob-4b78edd1b5baad75/tmp/PageRank_graph_prep_1.cloudera.20160716.171336.184035/files/...
Adding our job to existing cluster j-2TFUMR7W2PF1V
Waiting for step 1 of 1 (s-3DL634XJO222Y) to complete...
  Opening ssh tunnel to resource manager...
  Connect to resource manager at: http://localhost:40247/cluster
  RUNNING for 8.5s
   100.0% complete
  RUNNING for 43.4s
     5.0% complete
  RUNNING for 74.7s
    80.0% complete
  COMPLETED
Attempting to fetch counters from logs...
Looking for step log in /mnt/var/log/hadoop/steps/s-3DL634XJO222Y on ec2-54-149-192-241.us-west-2.compute.amazonaws.com...
  Parsing step log: ssh://ec2-54-149-192-241.us-west-2.compute.amazonaws.com/mnt/var/log/hadoop/steps/s-3DL634XJO222Y/syslog
Counters: 54
	File Input Format Counters 
		Bytes Read=756

In [303]:
%reload_ext autoreload
%autoreload 2

from numpy import random,array
from PageRank_step1 import MRpageRank1
import ast
import os

## Setting up input and output folders for 
## the iterative algorithm
# !rm -r /home/cloudera/w261/HW9/data/toy_prepared_graph_input
# !rm -r /home/cloudera/w261/HW9/data/toy_prepared_graph_output
# !mkdir /home/cloudera/w261/HW9/data/toy_prepared_graph_input
# !rm -r /home/cloudera/w261/HW9/data/toy_directed_graph_input

!aws s3 rm s3://cerc-w261/HW9/toy/prepared_graph_input --recursive --quiet
!aws s3 rm s3://cerc-w261/HW9/toy/prepared_graph_output --recursive --quiet

# !./PageRank_graph_prep_1.py -r hadoop /home/cloudera/w261/HW9/data/toy/example_1.txt \
# > /home/cloudera/w261/HW9/data/toy_prepared_graph_input/input.txt

!python PageRank_graph_prep_1.py \
-r emr s3://cerc-w261/HW9/toy/raw_input \
    --cluster-id=j-2TFUMR7W2PF1V \
    --aws-region=us-west-2 \
    --output-dir=s3://cerc-w261/HW9/toy/prepared_graph_input \
    --no-output
    
# !aws s3 rm s3://cerc-w261/HW7/synonyms_eda --recursive
# !python /home/cloudera/w261/HW7/src/synonyms_eda.py \
# -r emr s3://cerc-w261/HW7/synonyms_input \
#     --cluster-id=j-TK4RCWUIOMPL \
#     --aws-region=us-west-2 \
#     --output-dir=s3://cerc-w261/HW7/synonyms_eda \
#         --no-output

# !./PageRank_step1.py \
# /home/cloudera/w261/HW9/data/toy_prepared/input_1.txt \
# > /home/cloudera/w261/HW9/data/toy_prepared_graph_input/input.txt

## THIS SHIT AINT GONNA WORK CAUSE I WOULD NEED TO DOWNLOAD THE WHOLE
## GRAPH TO LOCAL TO OBTAIN THE NUMBER OF NODES IN THE GRAPH
## GONNA NEED TO WRITE NUM_NODES DIRECTLY TO S3 FROM THE PREPROCESSING
## JOB!!!!
input_dir = "/home/cloudera/w261/HW9/data/toy_prepared_graph_input/"

for name in os.listdir(input_dir):
    with open(input_dir+name) as f:
        num_nodes = 0
        for line in f:
            num_nodes += 1

i = 0
while(1):
#     frontier_exists = False
#     j = i + 1
    mr_job = MRpageRank1(args=["-r","emr",
                               "s3://cerc-w261/HW9/toy/prepared_graph_input",
                               "--alpha",0.15,
                               "--num-nodes", str(num_nodes), "--output-dir",
                               "s3://cerc-w261/HW9/toy/prepared_graph_output",
                              "--cluster-id=j-2TFUMR7W2PF1V",
                              "--aws-region=us-west-2"])
    print "iteration "+str(i)+" of Pagerank algorithm"
    graph = []
    with mr_job.make_runner() as runner: 
        runner.run()
    i = i + 1

#     !rm -r /home/cloudera/w261/HW9/data/toy_prepared_graph_input
#     !cp -rT /home/cloudera/w261/HW9/data/toy_prepared_graph_output /home/cloudera/w261/HW9/data/toy_prepared_graph_input
    
    !aws s3 rm s3://cerc-w261/HW9/toy/prepared_graph_input/ --recursive --quiet
    !aws s3 cp s3://cerc-w261/HW9/toy/prepared_graph_output s3://cerc-w261/HW9/toy/prepared_graph_input --recursive --quiet
    !aws s3 rm s3://cerc-w261/HW9/toy/prepared_graph_output --recursive --quiet
#     if not frontier_exists:
#         break
    if i >=20:
        break
# !cat /home/cloudera/w261/HW9/data/toy_prepared_graph_output/*

Using configs in /home/cloudera/.mrjob.conf
Using s3://mrjob-4b78edd1b5baad75/tmp/ as our temp dir on S3
Traceback (most recent call last):
  File "PageRank_graph_prep_1.py", line 62, in <module>
    MRpageRankGraphPrep.run()
  File "/home/cloudera/anaconda2/lib/python2.7/site-packages/mrjob/job.py", line 430, in run
    mr_job.execute()
  File "/home/cloudera/anaconda2/lib/python2.7/site-packages/mrjob/job.py", line 448, in execute
    super(MRJob, self).execute()
  File "/home/cloudera/anaconda2/lib/python2.7/site-packages/mrjob/launch.py", line 160, in execute
    self.run_job()
  File "/home/cloudera/anaconda2/lib/python2.7/site-packages/mrjob/launch.py", line 230, in run_job
    runner.run()
  File "/home/cloudera/anaconda2/lib/python2.7/site-packages/mrjob/runner.py", line 473, in run
    self._run()
  File "/home/cloudera/anaconda2/lib/python2.7/site-packages/mrjob/emr.py", line 862, in _run
    self._launch()
  File "/home/cloudera/anaconda2/lib/python2.7/site-packages/mrjob/em

AssertionError: Input path s3://cerc-w261/HW9/toy/prepared_graph_input does not exist!

In [143]:
!cat /home/cloudera/w261/HW9/data/toy_prepared/input.txt

"1"	["", 0.090909090909090912]
"10"	["5", 0.090909090909090912]
"11"	["5", 0.090909090909090912]
"2"	["3", 0.090909090909090912]
"3"	["2", 0.090909090909090912]
"4"	["1,2", 0.090909090909090912]
"5"	["2,4,6", 0.090909090909090912]
"6"	["2,5", 0.090909090909090912]
"7"	["2,5", 0.090909090909090912]
"8"	["2,5", 0.090909090909090912]
"9"	["2,5", 0.090909090909090912]


In [229]:
!./PageRank_step1.py -r hadoop \
/home/cloudera/w261/HW9/data/toy_prepared/input_1.txt #> \
# /home/cloudera/w261/HW9/data/toy_prepared/input_2.txt

Using configs in /home/cloudera/.mrjob.conf
Creating temp directory /tmp/PageRank_step1.cloudera.20160716.044136.325462
Looking for hadoop binary in $PATH...
Found hadoop binary: /usr/bin/hadoop
Using Hadoop version 2.6.0
Copying local files to hdfs:///user/cloudera/tmp/mrjob/PageRank_step1.cloudera.20160716.044136.325462/files/...
Looking for Hadoop streaming jar in /home/hadoop/contrib...
Looking for Hadoop streaming jar in /usr/lib/hadoop-mapreduce...
Found Hadoop streaming jar: /usr/lib/hadoop-mapreduce/hadoop-streaming.jar
Running step 1 of 1...
  packageJobJar: [] [/usr/jars/hadoop-streaming-2.6.0-cdh5.7.0.jar] /tmp/streamjob8553147381462418970.jar tmpDir=null
  Connecting to ResourceManager at quickstart.cloudera/10.0.2.15:8032
  Connecting to ResourceManager at quickstart.cloudera/10.0.2.15:8032
  Total input paths to process : 1
  number of splits:1
  Submitting tokens for job: job_1468516795796_0056
  Submitted application application_1468516795796_0056
  The url to track the

In [230]:
!cat /home/cloudera/w261/HW9/data/toy_prepared/input_2.txt

In [114]:
## first we spool up a cluster
!mrjob create-cluster \
--max-hours-idle 1 \
--aws-region=us-west-2 -c ~/.mrjob.conf

Unexpected option hadoop from /home/cloudera/.mrjob.conf
Using s3://mrjob-4b78edd1b5baad75/tmp/ as our temp dir on S3
Creating persistent cluster to run several jobs in...
Creating temp directory /tmp/no_script.cloudera.20160714.031502.443347
Copying local files to s3://mrjob-4b78edd1b5baad75/tmp/no_script.cloudera.20160714.031502.443347/files/...
j-17BJ3A7Y7825Q


In [146]:
!aws s3 rm s3://cerc-w261/HW9/toy/output --recursive
!rm -r /home/cloudera/w261/HW9/data/toy-output
!python PageRank_graph_prep_1.py \
-r emr /home/cloudera/w261/HW9/data/toy/example.txt \
    --cluster-id=j-17BJ3A7Y7825Q \
    --aws-region=us-west-2 \
    --output-dir=s3://cerc-w261/HW9/toy/output

delete: s3://cerc-w261/HW9/toy/output/_SUCCESS
delete: s3://cerc-w261/HW9/toy/output/part-00005 
delete: s3://cerc-w261/HW9/toy/output/part-00006 
delete: s3://cerc-w261/HW9/toy/output/part-00003 
delete: s3://cerc-w261/HW9/toy/output/part-00001 
delete: s3://cerc-w261/HW9/toy/output/part-00004 
delete: s3://cerc-w261/HW9/toy/output/part-00007 
delete: s3://cerc-w261/HW9/toy/output/part-00000 
delete: s3://cerc-w261/HW9/toy/output/part-00002 
rm: cannot remove `/home/cloudera/w261/HW9/data/toy-output': No such file or directory
Using configs in /home/cloudera/.mrjob.conf
Unexpected option hadoop from /home/cloudera/.mrjob.conf
Using s3://mrjob-4b78edd1b5baad75/tmp/ as our temp dir on S3
Creating temp directory /tmp/PageRank_graph_prep.cloudera.20160714.043202.503127
Copying local files to s3://mrjob-4b78edd1b5baad75/tmp/PageRank_graph_prep.cloudera.20160714.043202.503127/files/...
Adding our job to existing cluster j-17BJ3A7Y7825Q
Waiting for step 1 of 1 (s-3M09ZY4ZZUA9U) to complete..

In [ ]:
## THINGS TO CONSIDER:
### Passed in N as a configuration parameter, that made that easy.
### def get_total_nodes(ofile):
#    ls_output = !hdfs dfs -ls $ofile
#    num_output_files = len(ls_output)-3
#    for i in range(num_output_files):
#        snum = str(i)
#        zeros = '0'*(5-len(snum))
#        rfile = ofile+'/part-'+zeros+snum
#        grep_output = !hdfs dfs -cat $rfile | grep '*total_nodes'
#        if len(grep_output) == 2:
#            _, total_nodes = grep_output[1].strip().split('\t')
#            break
#    return total_nodes
## COOL CODE:
#    def steps(self):
#        return ([MRStep(mapper=self.send_score, reducer=self.receive_score)] *
#                self.options.iterations)
## Here's my whole algorithm. Hope this helps:

# Mapper 1: if this node has outlinks, yield current pr/outdegree to each of them. If not, yeild ("dangling", mass). Also yeild the graph structure.
# Reducer 1: if the key is "dangling", sum all the values and yield (i, mass/N) for all values of i in the graph. Otherwise, sum the masses and yield the graph structure with the sum
# Reducer 2: add the mass to the number in the current graph and calulate the final pagerank for this iteration new_p = (1-df)*(1.0/N) + df*mass

In [2]:
## Drivers & Runners

In [3]:
## Run Scripts, S3 Sync

<h2 style="color:darkgreen">  HW 9.1 Analysis </h2>




<br><br>

<h2 style="color:red"> HW 9.2: Exploring PageRank teleportation and network plots </h2>

* In order to overcome  problems such as disconnected components, the damping factor (a typical value for d is 0.85) can be varied. 
* Using the graph in HW1, plot the test graph (using networkx, https://networkx.github.io/) for several values of the damping parameter alpha, so that each nodes radius is proportional to its PageRank score. 
* In particular you should do this for the following damping factors: [0,0.25,0.5,0.75, 0.85, 1]. 
* Note your plots should look like the following: https://en.wikipedia.org/wiki/PageRank#/media/File:PageRanks-Example.svg

<h2 style="color:darkgreen"> HW 9.2 Implementation </h2>

In [1]:
## Code goes here

In [2]:
## Drivers & Runners

In [3]:
## Run Scripts, S3 Sync

<h2 style="color:darkgreen">  HW 9.2 Analysis </h2>




<br><br>

<h2 style="color:red"> HW 9.3: Applying PageRank to the Wikipedia hyperlinks network </h2>

* Run your PageRank implementation on the Wikipedia dataset for 5 iterations, and display the top 100 ranked nodes (with alpha = 0.85).
* Run your PageRank implementation on the Wikipedia dataset for 10 iterations, and display the top 100 ranked nodes (with teleportation factor of 0.15).
* Have the top 100 ranked pages changed? Comment on your findings. 
* Plot the pagerank values for the top 100 pages resulting from the 5 iterations run. Then plot the pagerank values for the same 100 pages that resulted from the 10 iterations run.  


<h2 style="color:darkgreen"> HW 9.3 Implementation </h2>

In [1]:
## Code goes here

In [2]:
## Drivers & Runners

In [3]:
## Run Scripts, S3 Sync

 <h2 style="color:darkgreen">  HW 9.3 Analysis </h2>




<br><br>

<h2 style="color:red"> HW 9.4: Topic-specific PageRank implementation using MRJob </h2>

Modify your PageRank implementation to produce a topic specific PageRank implementation, as described in:

http://www-cs-students.stanford.edu/~taherh/papers/topic-sensitive-pagerank.pdf

Note in this article that there is a special caveat to ensure that the transition matrix is irreducible.   
This caveat lies in footnote 3 on page 3:
```
	A minor caveat: to ensure that M is irreducible when p
	contains any 0 entries, nodes not reachable from nonzero
	nodes in p should be removed. In practice this is not problematic.
```
and must be adhered to for convergence to be guaranteed.   

Run topic specific PageRank on the following randomly generated network of 100 nodes:

> s3://ucb-mids-mls-networks/randNet.txt (also available on Dropbox)

which are organized into ten topics, as described in the file:

> s3://ucb-mids-mls-networks/randNet_topics.txt  (also available on Dropbox)

Since there are 10 topics, your result should be 11 PageRank vectors (one for the vanilla PageRank implementation in 9.1, and one for each topic with the topic specific implementation). Print out the top ten ranking nodes and their topics for each of the 11 versions, and comment on your result. Assume a teleportation factor of 0.15 in all your analyses.

One final and important comment here:  please consider the requirements for irreducibility with topic-specific PageRank. In particular, the literature ensures irreducibility by requiring that nodes not reachable from in-topic nodes be removed from the network.

This is not a small task, especially as it it must be performed separately for each of the (10) topics.

So, instead of using this method for irreducibility, please comment on why the literature's method is difficult to implement, and what what extra computation it will require.   

Then for your code, please use the alternative, non-uniform damping vector:

```
vji = beta*(1/|Tj|); if node i lies in topic Tj

vji = (1-beta)*(1/(N - |Tj|)); if node i lies outside of topic Tj
```
for beta in (0,1) close to 1. 

With this approach, you will not have to delete any nodes. If beta > 0.5, PageRank is topic-sensitive, and if beta < 0.5, the PageRank is anti-topic-sensitive. For any value of beta irreducibility should hold, so please try beta=0.99, and perhaps some other values locally, on the smaller networks.

<h2 style="color:darkgreen"> HW 9.4 Implementation </h2>

In [1]:
## Code goes here

In [2]:
## Drivers & Runners

In [3]:
## Run Scripts, S3 Sync

<h2 style="color:darkgreen">  HW 9.4 Analysis </h2>




<br><br>

<center><div class='jumbotron'><h3 style='color:darkblue'>---------  OPTIONAL QUESTIONS SECTION --------</h3></div></center>

<h2 style="color:red"> HW 9.5: (OPTIONAL) Applying topic-specific PageRank to Wikipedia</h2>

Here you will apply your topic-specific PageRank implementation to Wikipedia, defining topics (very arbitrarily) for each page by the length (number of characters) of the name of the article mod 10, so that there are 10 topics. 

* Once again, print out the top ten ranking nodes and their topics for each of the 11 versions, and comment on your result. Assume a teleportation factor of 0.15 in all your analyses. Run for 10 iterations.
* Plot the pagerank values for the top 100 pages resulting from the 5 iterations run in HW 9.3. 
* Then plot the pagerank values for the same 100 pages that result from the topic specific pagerank after 10 iterations run. 
* Comment on your findings. 

<h2 style="color:darkgreen"> HW 9.5 Implementation </h2>

In [1]:
## Code goes here

In [2]:
## Drivers & Runners

In [3]:
## Run Scripts, S3 Sync

<h2 style="color:darkgreen">  HW 9.5 Analysis </h2>




<br><br>

<h2 style="color:red"> HW 9.6:  (OPTIONAL) TextRank</h2>

* What is TextRank? Describe the main steps in the algorithm. Why does TextRank work?
* Implement TextRank in MrJob for keyword phrases (not just unigrams) extraction using co-occurrence based similarity measure with with sizes of N = 2 and 3. And evaluate your code using the following example using precision, recall, and FBeta (Beta=1):
```
"Compatibility of systems of linear constraints over the set of natural numbers
Criteria of compatibility of a system of linear Diophantine equations, strict 
inequations, and nonstrict inequations are considered. Upper bounds for
components of a minimal set of solutions and algorithms of construction of 
minimal generating sets of solutions for all types of systems are given. 
These criteria and the corresponding algorithms for constructing a minimal 
supporting set of solutions can be used in solving all the considered types of 
systems and systems of mixed types." 
```
* The extracted keywords should in the following set:
```
linear constraints, linear diophantine equations, natural numbers, non-strict inequations, strict inequations, upper bounds
```

<h2 style="color:darkgreen"> HW 9.6 Implementation </h2>

In [1]:
## Code goes here

In [2]:
## Drivers & Runners

In [3]:
## Run Scripts, S3 Sync

<h2 style="color:darkgreen">  HW 9.6 Analysis </h2>




<br><br>

<center><div class='jumbotron'><h2 style='color:green'>-------  END OF HWK 9 --------</h2></div></center>